In [1]:
# ============================================================
# 操作変数法（Instrumental Variables, IV）による因果効果推定のデモ
# ============================================================
# このスクリプトは、シンプルな線形モデル
#
#   結果変数:  Y
#   処置変数:  T
#   操作変数:  Z
#
# に対して 2 段階最小二乗法（2SLS, IV 推定量）を適用する例になっている。
# statsmodels の sandbox モジュールにある IV2SLS を用いて、
# 「エラー項と相関のある可能性がある T を、外生な Z で“置き換える”」
# という操作変数法のアイデアを実装している。
#
# ※このコードのデータ生成過程では、T は u と独立に生成されているため
#   「T が本当に内生的」という状況は作られていない。
#   そのため理論的には OLS と同じような推定結果になるはずであり、
#   IV を使うメリットはない。あくまで「IV 推定の書き方」のデモである点に注意。

# ------------------------------------------------------------
# 使用するライブラリのインポート
# ------------------------------------------------------------
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS  # 2SLS（操作変数法）推定器

# ------------------------------------------------------------
# 乱数シードの設定（再現性確保）
# ------------------------------------------------------------
np.random.seed(0)

# ------------------------------------------------------------
# サンプルデータの生成（DGP: Data Generating Process）
# ------------------------------------------------------------
n = 1000  # 観測値の数

# 操作変数 Z（instrument）:
#  - 処置 T には影響を与えるが、
#  - 結果 Y には T を通じてのみ影響する（排除制約）ことを想定する。
# ここでは単純に N(0,1) からサンプル。
Z = np.random.normal(size=n)

# 誤差項 u（outcome のノイズ）:
#  - Y の決定式に入る「観測されない要因」を表す。
#  - 理想的な IV の設定では、Z と独立であること（外生性）が重要。
u = np.random.normal(size=n)

# 処置変数 T（treatment）:
#  - Z の関数として生成することで、「Z と T の相関（relevance）」を確保する。
#  - ここでは T = Z + ε という形の単純な線形モデル。
T = Z + np.random.normal(size=n)

# 結果変数 Y（outcome）:
#  - 真の構造モデルを Y = T + u として定義。
#  - 理想的には「T の係数 1」が因果効果となる設定。
#  - 今回の DGP では T と u は独立なので、T は本来“外生的（exogenous）”であり、
#    OLS でも一貫推定が可能なケースになっている。
Y = T + u

# ------------------------------------------------------------
# DataFrame にまとめる（確認・拡張を容易にするため）
# ------------------------------------------------------------
df = pd.DataFrame({"Y": Y, "T": T, "Z": Z, "u": u})

# ------------------------------------------------------------
# statsmodels 用に説明変数・被説明変数を整形
# ------------------------------------------------------------

# 結果変数 Y：
#  - 被説明変数として Series をそのまま利用する。
Y = df["Y"]

# 処置変数 T：
#  - 内生的かもしれない説明変数（endogenous regressor）として扱う。
#  - sm.add_constant により切片（定数項）を列として追加し、
#    回帰モデルに「β0 + β1 T」の形を持たせる。
T = sm.add_constant(df["T"])

# 操作変数 Z：
#  - instrument として用いる変数。
#  - ここでは Z と定数項を含む行列 [1, Z] を instrument set として指定する。
#  - 理論上は
#      1. Relevance:   Cov(Z, T) ≠ 0（T と十分に相関）
#      2. Exogeneity:  Cov(Z, u) = 0（誤差項 u とは無相関）
#      3. Exclusion:   Z は T を通じてのみ Y に影響
#    の 3 条件を満たす必要がある。
Z = sm.add_constant(df["Z"])

# ------------------------------------------------------------
# 操作変数法（2SLS, IV2SLS）を適用
# ------------------------------------------------------------
# IV2SLS(Y, X, Z) の引数：
#   - Y: 被説明変数（outcome）
#   - X: 説明変数行列（内生変数 T と定数項など）
#   - Z: 操作変数行列（instrument set：Z と定数項など）
#
# 内部的な 2 段階のイメージ：
#   1st stage:
#       T = π0 + π1 Z + v
#      → Z から T の「IV 予測値」 T_hat を作る。
#   2nd stage:
#       Y = β0 + β1 T_hat + ε
#      → β1 を T の因果効果の推定量として解釈する。
#
# 今回の DGP では T と u が独立なので、実は「T は内生ではない」。
# そのため IV 推定量 β1 は OLS 推定量と同じ（あるいは近い）値になり、
# むしろ効率（分散）の面では OLS に劣る可能性すらある。
# もし「真に IV が必要なケース」を作りたい場合は、
#   Y = T + u, かつ T と u が相関するような DGP（例：T = Z + u + noise）
# に変更する必要がある。
iv = IV2SLS(Y, T, Z).fit()

# ------------------------------------------------------------
# 推定結果のサマリーを表示
# ------------------------------------------------------------
# summary() には、以下が含まれる：
#   - 推定された係数（coef）：
#       const（切片）と T の係数（因果効果の IV 推定値）
#   - 標準誤差（std err）、t 値、p 値
#   - 信頼区間（[0.025, 0.975]）
#   - 各種適合度指標（R-squared など）
#
# 理論上、DGP が Y = 1 * T + u なので、
#   ・T の推定係数は 1 に近い値
#   ・標準誤差はサンプルサイズ n=1000 に応じてそこまで大きくない
#   ・p 値は十分に小さく、ゼロ効果（β1=0）は棄却される
# という結果が期待される。
print(iv.summary())

                          IV2SLS Regression Results                           
Dep. Variable:                      Y   R-squared:                       0.652
Model:                         IV2SLS   Adj. R-squared:                  0.651
Method:                     Two Stage   F-statistic:                     902.4
                        Least Squares   Prob (F-statistic):          9.68e-142
Date:                Fri, 12 Dec 2025                                         
Time:                        14:47:26                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0105      0.031      0.342      0.7